In [39]:
import pyterrier as pt
import pickle
import pandas as pd
import shutil
import os
from pathlib import Path
import re

In [14]:
dataset = pt.get_dataset('irds:beir/fiqa/test')

In [15]:
for doc in dataset.get_corpus_iter():
    print(doc.keys())
    print(doc)
    break


beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

dict_keys(['text', 'docno'])
{'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.", 'docno': '3'}


In [16]:
dataset.get_corpus_iter()

In [17]:
queries = dataset.get_topics()
display(queries)

,qid,query
0,4641,Where should I park my rainy-day / emergency f...
1,5503,Tax considerations for selling a property belo...
2,7803,Can the Delta be used to calculate the option ...
3,7017,Basic Algorithmic Trading Strategy
4,10152,What does a high operating margin but a small ...
...,...,...
643,4102,How can I determine if my rate of return is “g...
644,3566,Where can I buy stocks if I only want to inves...
645,94,Using credit card points to pay for tax deduct...
646,2551,How to find cheaper alternatives to a traditio...


In [18]:
qrels = dataset.get_qrels()
display(qrels)

,qid,docno,label,iteration
0,8,566392,1,0
1,8,65404,1,0
2,15,325273,1,0
3,18,88124,1,0
4,26,285255,1,0
...,...,...,...,...
1701,11039,330058,1,0
1702,11039,91183,1,0
1703,11054,155053,1,0
1704,11054,321015,1,0


In [24]:
index_path =  str(Path("./beir-index").resolve())

In [25]:
os.path.exists(index_path)

True

In [26]:
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-21.0.9.10-hotspot" 

In [27]:
if os.path.exists(index_path):
    shutil.rmtree(index_path)
indexer= pt.index.IterDictIndexer(index_path, fields= True, meta={'docno':15}, text_attrs=['text'])

indexref = indexer.index(dataset.get_corpus_iter())

17:38:12.483 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (7915) - further warnings are suppressed


17:38:54.590 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 39 empty documents


In [28]:
index = pt.IndexFactory.of(indexref)
stats = index.getCollectionStatistics()

In [ ]:
print("Index folder:", index_path)
print("Number of documents:", stats.getNumberOfDocuments())
print("Number of postings:", stats.getNumberOfPostings())
print("Number of tokens:", stats.getNumberOfTokens())
print("Number of unique terms:", stats.getNumberOfUniqueTerms())
print(f"Average document length:{stats.getAverageDocumentLength():.2f}")


Index folder: C:\Users\nurpe\OneDrive\Desktop\Financial_search_engine\var\beir-index
Number of documents: 57638
Number of postings: 2714611
Number of tokens: 3783214
Number of unique terms: 51260
Average document length:65.64


In [29]:
q_text = dataset.get_topics("text")
q_text

,qid,query
0,4641,Where should I park my rainy-day / emergency f...
1,5503,Tax considerations for selling a property belo...
2,7803,Can the Delta be used to calculate the option ...
3,7017,Basic Algorithmic Trading Strategy
4,10152,What does a high operating margin but a small ...
...,...,...
643,4102,How can I determine if my rate of return is “g...
644,3566,Where can I buy stocks if I only want to inves...
645,94,Using credit card points to pay for tax deduct...
646,2551,How to find cheaper alternatives to a traditio...


In [30]:
counts = qrels.groupby("qid")["docno"].count()  # group by query id and count documents
print("Overall Statistics")
print(counts.describe())

Overall Statistics
count    648.000000
mean       2.632716
std        2.062616
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       15.000000
Name: docno, dtype: float64


In [36]:
bm25 = pt.terrier.Retriever(indexref, wmodel='BM25', controls={"qtype": "matchop"})
tfidf = pt.terrier.Retriever(indexref, wmodel="TF_IDF", controls={"qtype": "matchop"})

In [40]:
def rmv_problems(text: str) -> str:
    text = str(text)

    text = text.replace(":", " ")
    text = text.replace("'", " ")
    text = text.replace('"', " ")
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [41]:
q_text['query'] = q_text['query'].apply(rmv_problems)

In [42]:
res_bm25 = bm25.transform(q_text[['qid','query']])
res_tfidf = tfidf.transform(q_text[['qid','query']])

In [46]:
res_bm25

,qid,docid,docno,rank,score,query
0,4641,36224,376148,0,41.677305,Where should I park my rainy-day / emergency f...
1,4641,47916,497993,1,29.149791,Where should I park my rainy-day / emergency f...
2,4641,55690,580025,2,26.773005,Where should I park my rainy-day / emergency f...
3,4641,24501,253614,3,26.640181,Where should I park my rainy-day / emergency f...
4,4641,3157,32833,4,24.265187,Where should I park my rainy-day / emergency f...
...,...,...,...,...,...,...
645148,2399,25037,259223,995,10.230066,Where do web sites get foreign exchange curren...
645149,2399,33542,348029,996,10.229760,Where do web sites get foreign exchange curren...
645150,2399,13368,137444,997,10.221526,Where do web sites get foreign exchange curren...
645151,2399,22944,237573,998,10.219489,Where do web sites get foreign exchange curren...


In [48]:
from pyterrier.measures import *

bm25_evaluated = pt.Evaluate(res_bm25, dataset.get_qrels(), metrics=[AP@10,P@5,P@10], perquery=False)
tfidf_evaluated = pt.Evaluate(res_tfidf, dataset.get_qrels(), metrics=[AP@10,P@5,P@10], perquery=False)


In [49]:
bm25_evaluated

{'AP@10': 0.19396587696466464,
 'P@5': 0.10648148148148186,
 'P@10': 0.07037037037037075}

In [51]:
tfidf_evaluated

{'AP@10': 0.19392245231599844,
 'P@5': 0.10864197530864238,
 'P@10': 0.07114197530864234}

In [54]:
print(shutil.which("perl"))

None


In [56]:
os.environ["PATH"] = r"C:\Strawberry\perl\bin;" + os.environ["PATH"]

In [57]:
pt.Experiment(
    [tfidf, bm25],
    dataset.get_topics('text'),
    dataset.get_qrels(),
    eval_metrics=["map"],
    names=["TF_IDF", "BM25"],
    baseline=0, # For statistical significance testing
    verbose=True,
)

20:08:03.105 [main] ERROR org.terrier.querying.LocalManager -- Problem running Matching, returning empty result set as query 9329
java.io.IOException: Unknown field BROKERS - known fields are [text]
	at org.terrier.matching.matchops.SingleTermOp.getPostingIterator(SingleTermOp.java:120)
	at org.terrier.matching.matchops.SingleTermOp.getMatcher(SingleTermOp.java:149)
	at org.terrier.matching.PostingListManager.<init>(PostingListManager.java:304)
	at org.terrier.matching.PostingListManager.<init>(PostingListManager.java:282)
	at org.terrier.matching.daat.Full.match(Full.java:89)
	at org.terrier.querying.LocalManager$ApplyLocalMatching.process(LocalManager.java:518)
	at org.terrier.querying.LocalManager.runSearchRequest(LocalManager.java:895)
20:08:07.063 [main] ERROR org.terrier.querying.LocalManager -- Problem running Matching, returning empty result set as query 2384
java.io.IOException: Unknown field TAX - known fields are [text]
	at org.terrier.matching.matchops.SingleTermOp.getPosti

pt.Experiment:   0%|          | 0/2 [00:13<?, ?system/s]


JavaException: JVM exception occurred: org.terrier.querying.parser.QueryParserException: Failed to process qid 4681 'How to fix Finance::Quote to pull quotes in GnuCash' -- Encountered "" at line 1, column 20.

org.terrier.querying.parser.QueryParserException: Failed to process qid 4681 'How to fix Finance::Quote to pull quotes in GnuCash' -- Encountered "" at line 1, column 20.

	org.terrier.querying.parser.QueryParser.parseQuery(QueryParser.java:81)
	org.terrier.querying.parser.QueryParser.parseQuery(QueryParser.java:50)
	org.terrier.querying.TerrierQLParser.process(TerrierQLParser.java:39)
	org.terrier.querying.LocalManager.runSearchRequest(LocalManager.java:895)
	Caused by:
	org.terrier.querying.parser.ParseException: Encountered "" at line 1, column 20.

	org.terrier.querying.parser.TerrierQLParser.generateParseException(TerrierQLParser.java:1088)
	org.terrier.querying.parser.TerrierQLParser.jj_consume_token(TerrierQLParser.java:959)
	org.terrier.querying.parser.TerrierQLParser